In [2]:
%pip install torchvision
%pip install -U scikit-learn

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached torchvision-0.12.0-cp37-cp37m-manylinux1_x86_64.whl (21.0 MB)
  Using cached torch-1.11.0-cp37-cp37m-manylinux1_x86_64.whl (750.6 MB)
Note: you may need to restart the kernel to use updated packages.
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptograp

# Getting Data
1. gets data from torchvision (CIFAR10 dataset)
2. data is in 32x32x3 RGB pytorch tensors

In [4]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch
import numpy as np
train_data = datasets.CIFAR10(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)

test_data = datasets.CIFAR10(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)
print(np.isinf(train_data.data).any())

Files already downloaded and verified
False


# Running PCA transform
1. Split data into RGB
2. Run a PCA on each image in RGB
3. Reshape projections to square images
4. Run inverse transform
5. Reshape to original size images
### Remarks:
1. Lower n_components yields faster performance by PCA
    - Will log time complexities here:

In [ ]:
from PCA_utils import complete_transform
N_COMPONENTS = 4*4
reconstructed_train_images, T_pca, projection_images = complete_transform(train_data, n_components=N_COMPONENTS)
reconstructed_test_images, T, projection_test_images = complete_transform(test_data, n_components=N_COMPONENTS)
%time

# Visualize original images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# original images
fig = plt.figure(figsize=(10,10))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
for i in range(10):
    ax = fig.add_subplot(5, 5, i+1, xticks=[], yticks=[])
    ax.imshow(train_data.data[i], cmap=plt.cm.bone, interpolation='nearest')
plt.show()

# Visualize intermediate states

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# projections
fig = plt.figure(figsize=(10,10))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
max_val = np.max(projection_images)
print(max_val)
# ratio of max val to 255
ratio = max_val/255
for i in range(10):
    ax = fig.add_subplot(5, 5, i+1, xticks=[], yticks=[])
    ax.imshow((projection_images[i]/ratio).astype('uint8'), cmap=plt.cm.bone, interpolation='nearest')
plt.show()

# Inverse transform images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# reconstructed images
fig = plt.figure(figsize=(10,10))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
max_val = np.max(reconstructed_train_images)
print(max_val)
# ratio of max val to 255
ratio = max_val/255
for i in range(10):
    ax = fig.add_subplot(5, 5, i+1, xticks=[], yticks=[])
    # round to nearest int
    img = (reconstructed_train_images[i]/ratio).astype('uint8')
    ax.imshow(img.astype('uint8'), cmap=plt.cm.bone, interpolation='nearest')

plt.show()

# Get Variance Scores

In [ ]:
print("Red: " , r_pca.explained_variance_ratio_.sum())
print("Green: ", g_pca.explained_variance_ratio_.sum())
print("Blue: ", b_pca.explained_variance_ratio_.sum())

"""
8*8:
    Red: .
    Green: .
    Blue: .
12*12:
    Red: .
    Green: .
    Blue: .
16*16:
    Red: .972
    Green: .973
    Blue: .975
20*20
    Red: .
    Green: .
    Blue: .
24*24
    Red: .
    Green: .
    Blue: .
28*28
    Red: .
    Green: .
    Blue: .
"""

# Upload images for training and evaluation

In [ ]:
# convert numpy arrays to tensors
train_images = torch.from_numpy(reconstructed_train_images).float()
test_images = torch.from_numpy(reconstructed_test_images).float()
# save data
torch.save(train_images, "pca_train_images_{}.pt".format(N_COMPONENTS))
torch.save(train_data.targets, "pca_train_labels.pt")
torch.save(test_images, "pca_test_images_{}.pt".format(N_COMPONENTS))
torch.save(test_data.targets, "pca_test_labels.pt")